In [12]:
import cvxpy as cp
import numpy as np
import csv

In [13]:
def DCOPF(Y,PGl,PGu,PD,thetaL,thetaU,CQ,CL,PF,slack,gens_to_buses):
    num_buses = len(Y)
    num_generators = PGl.shape[1]
    B = np.imag(Y)
    t = len(PD)

    PG = cp.Variable((t,num_generators)); # optimization variable: PG_i
    theta = cp.Variable((t,num_buses)); # optimization variable: theta_i

    objective = cp.Minimize(cp.sum(PG**2 * CQ.T + PG * CL.T)) # objective function

    constraints = [
        # Power generation constraints
        PG - PGu <= 0,
        -PG + PGl <= 0,

        #Bus phase angle constraints
        theta.T - thetaU <= 0,
        -theta.T + thetaL <= 0,

        #Power balance constraint
        PG * gens_to_buses == theta * B + PD,

        #Slack angle constraint   
        theta[:,slack] == 0
    ]
    
    # Line capacity constraints
    count = 0
    for i in range(num_buses):          
        for j in range(i+1, num_buses):
            constraints.append(B[i,j] * (theta[:,i] - theta[:,j]) - PF[:,count] <= 0)
            constraints.append(B[i,j] * (theta[:,j] - theta[:,i]) - PF[:,count] <= 0)
            count = count + 1

    # SOLVE IT
    prob = cp.Problem(objective, constraints)
    result = prob.solve()
    print(result)
    
    pf_opt = np.zeros((t,num_buses,num_buses))
    for i in range(num_buses):
        for j in range(i+1, num_buses):
            pf_opt[:,i,j] = B[i,j] * (theta[:,i].value - theta[:,j].value)
    
    cost = objective.value
    p_opt = PG.value
    theta_opt = theta.value
    lmp_opt = constraints[4].dual_value.reshape((t,num_buses))
    
    return pf_opt, cost, p_opt, theta_opt, lmp_opt

In [22]:
def load_generators(filename, generator_dict):
    generators_file = open(filename)
    reader = csv.reader(generators_file)
    next(reader)
    count = 0
    
    while True:
        try:
            next_line = next(reader)
        except StopIteration:
            break
            
        fuel = next_line[0]
        weather_zone = next_line[1]
        jan_mar = float(next_line[2])
        apr_sep = float(next_line[3])
        oct_dec = float(next_line[4])
        gen_id = "{} {}".format(weather_zone, fuel)
        
        generator_dict[gen_id] = {
                        'idx' : count,
                        'bus' : weather_zone,
                        'p_nom' : apr_sep,
                        'cq' : 1,
                        'cl' : 1,
        }
        count += 1
        
        if 'wind' in fuel:
            generator_dict[gen_id]['p_max_pu'] = wind_curve
        else:
            generator_dict[gen_id]['p_max_pu'] = 1
            
    generators_file.close()

In [31]:
NUM_HOURS = 31*24
NUM_BUSES = 9

# Useful data structures to transfer between bus indexes and names
regidxs = {"North" : 0,
         "West" : 1,
         "FarWest" : 2,
         "NorthCentral" : 3,
         "East" : 4,
         "SouthCentral" : 5,
         "South" : 6,
         "Coast" : 7,
         "Northwest" : 8}

regnames = ['North', 'West', 'FarWest', 'NorthCentral', 'East', 'SouthCentral', 'South', 'Coast', 'Northwest']

# distances of transmission lines, in km
distances = {}
distances[('FarWest', 'South')] = 579
distances[('FarWest', 'West')] = 224
distances[('West', 'North')] = 195
distances[('North', 'NorthCentral')] = 198
distances[('East', 'NorthCentral')] = 146
distances[('East', 'Coast')] = 290
distances[('West', 'SouthCentral')] = 340
distances[('SouthCentral', 'Coast')] = 243
distances[('NorthCentral', 'SouthCentral')] = 241 #note: needs fixing
distances[('South', 'SouthCentral')] = 193
distances[('South', 'Coast')] = 391
distances[('Northwest', 'North')] = 200 # made up

# Construct Y-bus
impedance_per_km = .01j

Y = np.zeros((NUM_BUSES, NUM_BUSES),dtype=complex)
for b1 in range(NUM_BUSES):
    for b2 in range(b1+1, NUM_BUSES):
        if (regnames[b1],regnames[b2]) in distances.keys():
            Y[b1,b2] = 1/(distances[(regnames[b1],regnames[b2])]*impedance_per_km)
        elif (regnames[b2],regnames[b1]) in distances.keys():
            Y[b1,b2] = 1/(distances[(regnames[b2],regnames[b1])]*impedance_per_km)
        else:
            Y[b1,b2] = 0
        Y[b2,b1] = Y[b1,b2]
    Y[b1,b1] = -1*np.sum(Y[b1,:])
    
# set voltage angle constraints
thetal = -np.ones((NUM_BUSES,NUM_HOURS))
thetau = np.ones((NUM_BUSES,NUM_HOURS))
    
# Get all the generators
generators_dict = {}
load_generators('zonal_generator_capacities.csv', generators_dict)
num_gens = len(generators_dict.keys())

# Gens to buses matrix
gens_to_buses = np.zeros((num_gens, NUM_BUSES))
for gen in generators_dict.keys():
    gen_idx = generators_dict[gen]['idx']
    bus_idx = regidxs[generators_dict[gen]['bus']]
    gens_to_buses[gen_idx, bus_idx] = 1
    
    
gens_to_buses = [[1,0,0],
                 [0,1,0],
                 [0,0,1]]

PGl = np.array([[0, 0, 0]]).repeat(NUM_HOURS, axis=0)
PGu = np.array([[1, 1, 0]]).repeat(NUM_HOURS, axis=0)
CQ = np.array([[1, 1, 0]])
CL = np.array([[1, 1, 0]])
PF = np.array([[1, 1, 1]]).repeat(NUM_HOURS, axis=0)
slack= 0
PD = np.array([[0, 0.3, 0.4]]).repeat(NUM_HOURS, axis=0)

In [32]:
pf_opt, cost, p_opt, theta_opt, lmp_opt = DCOPF(Y, PGl, PGu, PD, thetal, thetau, CQ, CL, PF, slack, gens_to_buses)
print('OPF:', pf_opt)
print('Total cost:', cost)
print('Generation:', p_opt)
print('Angles:', theta_opt)
print('LMPs: ', lmp_opt)

ValueError: Cannot broadcast dimensions  (744, 9) (744, 3)

In [34]:
NUM_HOURS = 31*24
NUM_BUSES = 3

Y = [[1j, -.5j, -.5j],
    [-.5j, 1j, -.5j],
    [-.5j, -.5j, 1j]]
    
# set voltage angle constraints
thetal = -np.ones((NUM_BUSES,NUM_HOURS))
thetau = np.ones((NUM_BUSES,NUM_HOURS))

gens_to_buses = [[1,0,0],
                 [0,1,0],
                 [0,1,0]]

PGl = np.array([[0, 0, 0]]).repeat(NUM_HOURS, axis=0)
PGu = np.array([[1, 1, 0]]).repeat(NUM_HOURS, axis=0)
CQ = np.array([[1, 1, 0]])
CL = np.array([[1, 1, 0]])
PF = np.array([[1, 1, 1]]).repeat(NUM_HOURS, axis=0)
slack= 0
PD = np.array([[0, 0.3, 0.4]]).repeat(NUM_HOURS, axis=0)

pf_opt, cost, p_opt, theta_opt, lmp_opt = DCOPF(Y, PGl, PGu, PD, thetal, thetau, CQ, CL, PF, slack, gens_to_buses)
print('OPF:', pf_opt)
print('Total cost:', cost)
print('Generation:', p_opt)
print('Angles:', theta_opt)
print('LMPs: ', lmp_opt)

351.53999999505817
OPF: [[[ 0.          0.01666667 -0.01666667]
  [ 0.          0.         -0.03333333]
  [ 0.          0.          0.        ]]

 [[ 0.          0.01666667 -0.01666667]
  [ 0.          0.         -0.03333333]
  [ 0.          0.          0.        ]]

 [[ 0.          0.01666667 -0.01666667]
  [ 0.          0.         -0.03333333]
  [ 0.          0.          0.        ]]

 ...

 [[ 0.          0.01666667 -0.01666667]
  [ 0.          0.         -0.03333333]
  [ 0.          0.          0.        ]]

 [[ 0.          0.01666667 -0.01666667]
  [ 0.          0.         -0.03333333]
  [ 0.          0.          0.        ]]

 [[ 0.          0.01666667 -0.01666667]
  [ 0.          0.         -0.03333333]
  [ 0.          0.          0.        ]]]
Total cost: 351.5400000039371
Generation: [[3.39424891e-11 3.50000000e-01 0.00000000e+00]
 [3.39424891e-11 3.50000000e-01 0.00000000e+00]
 [3.39424891e-11 3.50000000e-01 0.00000000e+00]
 ...
 [3.39424891e-11 3.50000000e-01 0.00000000e+00]